In [1]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import sys
sys.path.append('../../scripts')
import featurizer
sys.path.append('../../../../../scripts/preprocessing')
import preprocessing
import torch
from tqdm.notebook import tqdm
import warnings
from torch import nn
from typing import Union,Optional
from torch import Tensor
from torch.utils.data import DataLoader
from typing import List

In [2]:
df_btc = preprocessing.load_df('../../../../../../data/Binance/BTC_USDT.csv')

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



In [3]:
df_btc.head()

,timestamp,open,high,low,close,volume,returns
date,,,,,,,
2018-02-09 10:00:14.789,1518170414789,8230.46,8317.74,8230.45,8317.74,91.494282,NaN
2018-02-09 10:01:14.789,1518170474789,8304.00,8369.00,8304.00,8350.00,131.872916,0.003878
2018-02-09 10:02:14.789,1518170534789,8346.50,8380.00,8330.00,8380.00,84.544616,0.003593
2018-02-09 10:03:14.789,1518170594789,8379.00,8390.00,8348.00,8360.02,112.940304,-0.002384
2018-02-09 10:04:14.789,1518170654789,8350.04,8389.00,8348.00,8348.13,88.138218,-0.001422


In [4]:
warnings.filterwarnings('ignore')

path_btc = '../../../../../../data/data_new/raw/Binance/BTC_USDT/BTC_USDT.csv'

df_btc_feat = featurizer.load_and_featurize(path_btc)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



  0%|          | 0/1811 [00:00<?, ?it/s]

We make use of 90.44726670347875% of the data


0it [00:00, ?it/s]

In [5]:
path_eth = '../../../../../../data/data_new/raw/Binance/ETH_USDT/ETH_USDT.csv'

df_eth_feat = featurizer.load_and_featurize(path_eth)

Total NaN : 

timestamp    0
open         0
high         0
low          0
close        0
volume       0
returns      1
dtype: int64



  0%|          | 0/1811 [00:00<?, ?it/s]

We make use of 90.44726670347875% of the data


0it [00:00, ?it/s]

In [11]:
df_eth_feat.X.apply(lambda x : len(x)).value_counts()

124    1638
Name: X, dtype: int64

In [12]:
len(torch.tensor(df_btc_feat.X.tolist(),dtype = float))

1638

In [13]:
df_btc_feat.head()

,X,y,timestamp
0,"[0.034924874874129985, 0.01611405414408451, -0...",True,2018-02-16 23:59:00
1,"[-0.019750645849713644, 0.0038305703115333323,...",True,2018-02-17 23:59:00
2,"[0.014295249358891704, 0.005428320083860738, 0...",True,2018-02-18 23:59:00
3,"[-0.007939690941471955, 0.027906252400010434, ...",True,2018-02-19 23:59:00
4,"[0.030490725583469658, -0.0194371664240659, -0...",False,2018-02-20 23:59:00


In [14]:
df_eth_feat.head()

,X,y,timestamp
0,"[0.025933571490377005, 0.018530790269527753, -...",True,2018-02-16 23:59:00
1,"[-0.018415526273278915, 0.006147878217938052, ...",False,2018-02-17 23:59:00
2,"[0.008804971660111516, 0.005123795613559912, 0...",True,2018-02-18 23:59:00
3,"[-0.012066282953791307, 0.0213546255506607, 0....",True,2018-02-19 23:59:00
4,"[0.019955654101995624, -0.008939554612937384, ...",True,2018-02-20 23:59:00


In [15]:
from torch.utils.data import Dataset

class twoday_basic_dset(Dataset):
    
    def __init__(self,df):
        self.X = torch.tensor(df.X.tolist(),dtype = torch.float)
        self.y = torch.tensor(df.y.astype(int).tolist(),dtype = torch.float)
        
    def __getitem__(self,
                    index: int):
        return (self.X[index],self.y[index])
    
    def __len__(self):
        return len(self.X)

In [16]:
dset = twoday_basic_dset(df_btc_feat)

dloader = DataLoader(dset,
                     batch_size = 16,
                     num_workers= 64,
                     shuffle = True)


In [17]:
batch = next(iter(dloader))

In [18]:
batch

[tensor([[ 1.4721e-03, -5.5792e-03, -5.3914e-03,  ...,  5.9620e+07,
           2.1029e-02,  9.3930e-02],
         [ 2.3089e-03, -2.1265e-03, -4.6097e-03,  ...,  1.4298e+08,
           5.0294e-02,  1.3430e-01],
         [ 7.2443e-04,  3.1331e-03, -1.1476e-03,  ...,  4.8584e+07,
           1.6800e-01,  1.7438e-01],
         ...,
         [-5.8615e-03,  7.6257e-03,  2.6279e-03,  ...,  1.3653e+08,
           3.7112e-02,  1.0678e-01],
         [-8.0698e-03, -1.0560e-03, -2.8641e-03,  ...,  2.8180e+08,
           7.4219e-02,  9.7249e-02],
         [ 1.7956e-02, -1.1981e-03,  8.3647e-03,  ...,  2.3296e+08,
           5.6707e-02,  2.2576e-01]]),
 tensor([1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1.])]

In [19]:
class MLP(nn.Module):
    def __init__(
        self,
        embed_dims: Union[int, list],
        num_layers: Optional[int] = None,
        activation: str = "relu",
    ) -> None:
        super().__init__()

        if isinstance(embed_dims, int):
            assert num_layers is not None
            embed_dims = [embed_dims] * num_layers
        else:
            assert num_layers is None
            num_layers = len(embed_dims)

        assert activation == "relu"

        self.embed_dims = embed_dims
        self.num_layers = num_layers
        self.activation = activation

        self.layers = nn.ModuleList(
            [
                sublayer
                for i, (m, n) in enumerate(zip(embed_dims, embed_dims[1:]))
                for sublayer in [nn.Linear(m, n), nn.BatchNorm1d(n), nn.ReLU()]
                if not (i == num_layers - 2 and isinstance(sublayer, nn.ReLU))
            ]
        )


    def forward(self, x: Tensor) -> Tensor:
        for layer in self.layers:
            x = layer(x)
        return x

In [20]:
a = MLP([124,64,16])

In [60]:
#a(batch[0])

In [19]:
f = nn.Sequential(a,nn.Linear(16,1))

In [67]:
x = torch.tensor(df_btc_feat.X[0],dtype = torch.float)

In [68]:
a0 = nn.Linear(122,64)
a1 = nn.BatchNorm1d(64)

In [69]:
x = a0(x.view(-1,122))

In [75]:
x = torch.vstack([x,x])

In [78]:
df_btc_feat

,X,y,timestamp
0,"[0.48397252001717433, 0.4890715778328684, 0.53...",True,2018-02-15 23:59:00
1,"[0.6002010108536191, 0.5813901901235736, 0.554...",True,2018-02-16 23:59:00
2,"[0.5521041562319422, 0.5756853723931892, 0.556...",True,2018-02-17 23:59:00
3,"[0.6565078547973566, 0.6476409255223257, 0.657...",True,2018-02-18 23:59:00
4,"[0.39676013762499374, 0.4326060809664761, 0.40...",True,2018-02-19 23:59:00
...,...,...,...
1392,"[0.17751007558436646, 0.1770441927559447, 0.17...",True,2022-02-17 23:59:00
1393,"[0.19000899098333363, 0.20502533880136736, 0.1...",True,2022-02-18 23:59:00
1394,"[0.20746012365790603, 0.1879378522546066, 0.18...",False,2022-02-19 23:59:00
1395,"[0.16128656895314372, 0.1733759463626144, 0.17...",True,2022-02-20 23:59:00


In [25]:
list(a.children())[0]

ModuleList(
  (0): Linear(in_features=124, out_features=64, bias=True)
  (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=64, out_features=16, bias=True)
  (4): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [23]:
list(f.children())[1]

Linear(in_features=16, out_features=1, bias=True)

In [ ]:
self.layers = nn.ModuleList(
            [
                sublayer
                for i, (m, n) in enumerate(zip(embed_dims, embed_dims[1:]))
                for sublayer in [nn.Linear(m, n), nn.BatchNorm1d(n), nn.ReLU()]
                if not (i == num_layers - 2 and isinstance(sublayer, nn.ReLU))
            ]
        )

In [27]:
embed_dims = [3,4,5]
num_layers = 3

r = [sublayer for i, (m, n) in enumerate(zip(embed_dims, embed_dims[1:])) 
 for sublayer in [nn.Linear(m, n), nn.BatchNorm1d(n), nn.ReLU()] 
 if not (i == num_layers - 2 and isinstance(sublayer, nn.ReLU))
]

In [31]:
def ff(x):
    x += 1
    import pdb
    pdb.set_trace()
    x = x**2
    return x

In [71]:
def f(x:List[int]):
    return x[0]

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
import torchmetrics.functional.classification as tm
from lightning.pytorch.callbacks import Callback



class MLP_binary_classifier(pl.LightningModule):
    
    def __init__(self,
                 embed_dims: List[int],
                 lr = 0.001):
        super().__init__()
        self.mlp = MLP(embed_dims)
        self.Classifier_head = nn.Linear(embed_dims[-1],1)
        self.s = nn.Sigmoid()
        self.step_outputs = {"train": [],"val":[],"test":[]}
        self.lr = lr
        
    def forward(self,
                batch):
        
        x,y = batch
        x = self.mlp(x)
        x = self.Classifier_head(x)
        x = self.s(x)
        
        return x
          
    def _step(self,
              name: str,
              batch: List[torch.tensor]):
        
        target = batch[1].to(device)
        predicted_proba = self.forward(batch).flatten()
        criterion = nn.BCELoss()
        loss = criterion(predicted_proba,target)
        self.step_outputs[name].append({"loss": loss, "predicted_proba": predicted_proba, "target": batch[1]})
        
        self.log(
            f"{name}/loss", loss, prog_bar = True, on_epoch=True, logger = True, on_step= (name=="train"), batch_size=batch[1].shape[0]
        )
        
        
        return {"loss": loss, "predicted_proba": predicted_proba, "target": batch[1]}
    
    def training_step(self, batch, batch_idx = None):
        return self._step("train", batch)
    
    def validation_step(self, batch, batch_idx = None):
        return self._step("val", batch)
    
    def _epoch_end(self, name, outputs):
        y = torch.hstack([output["target"] for output in outputs])
        y = y.type(torch.long)
        preds = torch.hstack([output["predicted_proba"] for output in outputs])

        self.log(f"{name}/auroc", tm.binary_auroc(preds, y))
        self.log(f"{name}/average_precision", tm.binary_average_precision(preds, y))

        #precision, recall, _ = tm.binary_precision_recall_curve(preds, y)
        #for ten_x in range(1,11):
        #    x = ten_x / 10
        #    if compute_p_at_r(precision, recall, x) != None:
        #        self.log(f"{name}/p@r={x}", compute_p_at_r(precision, recall, x))
#
        #    y_pred = (preds > x).long()
        #    self.log(f"{name}/accuracy@thh={x}", tm.binary_accuracy(y_pred, y))
        #    self.log(f"{name}/recall@thh={x}", tm.binary_recall(y_pred, y))
        #    self.log(f"{name}/precision@thh={x}", tm.binary_precision(y_pred, y))
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer  
    
    def on_training_epoch_end(self):
        outputs = self.step_outputs['train']
        return self._epoch_end("train", outputs)
    
    def on_validation_epoch_end(self):
        outputs = self.step_outputs['val']
        return self._epoch_end("val", outputs)
    
    #def train_dataloader(self):
    #    dloader = dd.get_dataloader(self.batch_size,self.trainset,self.esm_feat,is_inference = False)
    #    return dloader
    #
    #def val_dataloader(self):
    #    dloader = dd.get_dataloader(self.batch_size,self.valset,self.esm_feat,is_inference = True)
    #    return dloader
            

In [24]:
class MyCallback(pl.Callback):
    
    def on_x_epoch_end(self, name, pl_module):
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(pl_module.step_outputs[name]).mean()
        pl_module.log(f"{name}_epoch_mean", epoch_mean)
        # free up the memory
        pl_module.step_outputs[name].clear()
        
    def on_train_epoch_end(self, trainer, pl_module):
        return self.on_x_epoch_end("train", pl_module)
        
    def on_validation_epoch_end(self, trainer, pl_module):
        return self.on_x_epoch_end("val", pl_module)

In [31]:
df_btc_feat[:int(.6*1638)]

,X,y,timestamp
0,"[0.034924874874129985, 0.01611405414408451, -0...",True,2018-02-16 23:59:00
1,"[-0.019750645849713644, 0.0038305703115333323,...",True,2018-02-17 23:59:00
2,"[0.014295249358891704, 0.005428320083860738, 0...",True,2018-02-18 23:59:00
3,"[-0.007939690941471955, 0.027906252400010434, ...",True,2018-02-19 23:59:00
4,"[0.030490725583469658, -0.0194371664240659, -0...",False,2018-02-20 23:59:00
...,...,...,...
977,"[0.026602033094025535, 0.01361562492528634, -0...",True,2021-02-28 23:59:00
978,"[0.016678208075497025, -0.005895291746056963, ...",True,2021-03-01 23:59:00
979,"[-0.012602570478268849, 0.018282083145440664, ...",True,2021-03-02 23:59:00
980,"[0.03209162590296177, -0.007095394097018981, 0...",True,2021-03-03 23:59:00


In [25]:
embed_dims = [124,64,16]
mlp = MLP(embed_dims)
Classifier_head = nn.Linear(embed_dims[-1],1)

In [26]:
criterion = nn.BCEWithLogitsLoss()
logits = mlp(batch[0])
logits = Classifier_head(logits).flatten()
target = batch[1]

In [124]:
s = nn.Sigmoid()
s(logits)

tensor([0.4771, 0.3650, 0.4252, 0.5285, 0.3797, 0.5968, 0.5490, 0.4507, 0.5741,
        0.5620, 0.4762, 0.5395, 0.4747, 0.5364, 0.2745, 0.3670],
       grad_fn=<SigmoidBackward0>)

In [125]:
criterion(logits,target)

tensor(0.7557, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [126]:
df_btc_feat.X.apply(lambda x : len(x)).value_counts()

124    1638
Name: X, dtype: int64

In [27]:
dloader = DataLoader(dset,
                     batch_size = 16,
                     num_workers= 64,
                     shuffle = True)

In [28]:
dset = twoday_basic_dset(df_btc_feat)

dloader = DataLoader(dset,
                     batch_size = 16,
                     num_workers= 64,
                     shuffle = True)

In [29]:
def get_dloader(df,
                batch_size,
                num_workers = 64,
                shuffle = True):
    
    dset = twoday_basic_dset(df)
    dloader = DataLoader(dset,batch_size = batch_size,num_workers = num_workers, shuffle = shuffle)
    
    return dloader

In [ ]:
train_df = df_btc_feat[:int(0.6*len(df_btc_feat))]
val_df = df_btc_feat[int(0.62*len(df_btc_feat)):int(0.75*len(df_btc_feat))]
test_df = df_btc_feat[int(0.75*len(df_btc_feat)):]

train_loader = get_dloader(train_df,16,64,True)
val_loader =  get_dloader(val_df,256,64,True)
test_df = get_dloader(test_df,256,64,True)

In [ ]:
model = MLP_binary_classifier([124,64,16])

In [ ]:
trainer = Trainer(accelerator="gpu")
trainer.fit(model,train_loader,val_loader)

In [83]:
#df_btc_feat.y.astype(int).tolist()

In [46]:
aa = list([np.array([12,2,3]),np.array([2,3,4])])

In [131]:
batch = next(iter(train_loader))

In [132]:
batch

(tensor([ 3.4925e-02,  1.6114e-02, -1.1200e-02,  1.0584e-02,  3.4774e-02,
         -1.4646e-02,  3.5579e-03, -8.6498e-05,  1.1311e-02,  2.8159e-03,
          4.3445e-03,  8.1577e-03, -1.9751e-02,  3.8306e-03, -1.4912e-02,
          4.5701e-03, -2.7489e-02,  1.1907e-02,  8.6070e-03, -1.3363e-02,
          1.5636e-02, -7.6774e-04, -4.2374e-03, -4.6191e-03,  1.4295e-02,
          5.4283e-03,  1.5616e-02,  2.8457e-03, -1.2084e-02,  1.8717e-02,
          3.3589e-02,  1.1188e-02, -3.9689e-03, -5.7774e-03,  5.1659e-03,
          1.7540e-02, -7.9397e-03,  2.7906e-02,  3.4680e-03,  1.4479e-02,
         -2.2927e-02, -1.9343e-03,  7.5471e-03,  1.7871e-02,  2.6416e-02,
          4.0287e-03, -2.3490e-02,  1.2847e-02,  3.0491e-02, -1.9437e-02,
         -4.9155e-03, -3.6452e-02,  1.6152e-02, -4.6277e-03,  3.1094e-03,
          2.1372e-02,  1.5339e-03, -2.4584e-03,  2.4266e-03,  1.0940e-02,
          3.3739e+07,  3.3717e+07,  2.0781e+07,  1.9839e+07,  4.7223e+07,
          3.2983e+07,  2.4010e+07,  3.

In [2]:
metrics = pd.read_csv('./lightning_logs/version_16/metrics.csv')

In [3]:
metrics

,train/loss_step,epoch,step,val/loss,train/loss_epoch
0,0.681599,0,49,NaN,NaN
1,NaN,0,61,0.548118,NaN
2,NaN,0,61,NaN,0.698738
3,0.752696,1,99,NaN,NaN
4,NaN,1,123,0.578921,NaN
5,NaN,1,123,NaN,0.638099
6,0.575539,2,149,NaN,NaN
7,NaN,2,185,0.553502,NaN
8,NaN,2,185,NaN,0.599506
9,0.622920,3,199,NaN,NaN
